# Import danych

In [ ]:
import pandas as pd
import os

DATA_FOLDER = './data_v2'
DATA_FILES = [os.path.join(DATA_FOLDER, file) for file in  os.listdir(DATA_FOLDER)]

datasets = {data: pd.read_json(data, lines=True) for data in DATA_FILES}

# Analiza danych

## Wartości null

In [ ]:
for dataset_name, dataset in datasets.items():
  null_dict = dataset.isnull().sum()
  print(f"DATASET {dataset_name}: ")
  print('---------------------------')
  for key, val in null_dict.items():
    print(f"{key:<15}: {val:<10}")
  print('===========================')
    

## IDS = -1

In [ ]:
for dataset_name, dataset in datasets.items():
  id_candidates = [key for key in dataset.keys() if 'id' in key]
  negative_ids = 0
  for id_candidate in id_candidates:
    negative_ids += len(dataset[dataset[id_candidate] == -1])
  print(f"DATASET {dataset_name}: ")
  print('---------------------------')
  print(f"Negative id\'s: {negative_ids:<10}")
  print('===========================')

## Wnioski

Wyraźnie widać, że wcześniejsze problemy z danymi już nie występują. Możemy zatem przejść do faktycznej analizy danych pod względem biznesowym

# Analiza danych

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
artists = pd.read_json("data_v2/artists.jsonl", lines=True)
sessions = pd.read_json("data_v2/sessions.jsonl", lines=True)
track_storage = pd.read_json("data_v2/track_storage.jsonl", lines=True)
tracks = pd.read_json("data_v2/tracks.jsonl", lines=True)
users = pd.read_json("data_v2/users.jsonl", lines=True)

pd.set_option('mode.chained_assignment', None)

### Typy zdarzeń

In [ ]:
sessions['event_type'].unique()

### Interesujące dla nas mogą być zdarzenia: 'PLAY' i 'SKIP'

Na ich podstawie możemy szacować czas słuchania wykonawcy muzycznego przez użytkowników. Zakładając, że zdarzenie PLAY oznacza średnio przesłuchanie całego utworu, a SKIP oznacza pominięcie średnio połowy długości utworu, możemy spróbować przedstawić średni czas słuchania wykonawcy w następujący sposób.

Okres dostępności danych

In [ ]:
print(f"Min: {sessions['timestamp'].min()} - Max: {sessions['timestamp'].max()}")
print(f"Period: {sessions['timestamp'].max() - sessions['timestamp'].min()}")

Mamy do czynienia z danymi z przedziału 4 miesięcy - od 9.01 do 10.04

Połączenie danych z utworami i artystami + segregacja po tygodniu wydarzenia

In [ ]:
def get_week(date):
  return date.week

merged = sessions.merge(tracks[['id', 'id_artist', 'duration_ms']], left_on='track_id', right_on='id', how='left')
merged = merged.merge(artists[['id', 'name']], left_on='id_artist', right_on='id', how='left')

# This will be removed in actual analysis
sessions_merged = merged[merged['timestamp'] < '2023-04-10']

sessions_merged['weeks_ordered'] = sessions_merged['timestamp'].apply(get_week)

In [ ]:
max_week = sessions_merged['weeks_ordered'].max()
min_week = sessions_merged['weeks_ordered'].min()

print(f"Min: {min_week}, Max: {max_week}")

## Wykresy akcji związanych z utworami w okresach

In [ ]:
plt.xlabel("Week")
plt.ylabel("Number of events")
for key in ['PLAY', 'SKIP']:
    plt.title(f"Number of {key} events per week")
    plt.plot(sessions_merged[sessions['event_type'] == key].groupby('weeks_ordered').size(), label=key, marker='o')
    plt.xticks(range(min_week, max_week + 1, 1))
    plt.show()


### Problemem może być niespójność, możemy przewidywać rokowania artystów, których utwory były słuchane w każdym z okresów

In [ ]:
all_artists = sessions_merged['name'].unique()
print(len(all_artists))

In [ ]:
consistent_artists = []

for artist in all_artists:
    if len(sessions_merged[sessions_merged['name'] == artist]['weeks_ordered'].unique()) == 12:
        consistent_artists.append(artist)

print(len(consistent_artists))

Mamy zaledwie 199 spośród 808 artystów, którzy byli słuchani w każdym okresie i jesteśmy dla nich w stanie przewidzieć rokowania za pomocą proponowanego rozwiązania.

In [ ]:
def approximate_playtime(artist_sessions):
    play_playtime = artist_sessions[artist_sessions['event_type'] == 'PLAY'].groupby('weeks_ordered')['duration_ms'].sum() * 1.66e-5 # convert to minutes
    # skip_playtime = artist_sessions[artist_sessions['event_type'] == 'SKIP'].groupby('weeks_ordered')['duration_ms'].sum() / 2
    return play_playtime

def plot_actions(artists, sessions_merged):
    COLS = len(artists)
    fig, axs = plt.subplots(1, COLS, figsize=(20, 5))
    
    col = 0
    for artist in artists:
        artist_sessions = sessions_merged[sessions_merged['name'] == artist]
        approximated_playtime = approximate_playtime(artist_sessions)
        axs[col].set_xlabel("Week [ordered Number]")
        axs[col].set_ylabel("Play time [minutes]")
        axs[col].plot(approximated_playtime, label=artist, marker='o')
        axs[col].set_title(f"Artist: {artist}")
        axs[col].grid()
        col += 1


### Sortowanie artystów po popularności

In [ ]:
artists_dic = {}
for artist in consistent_artists:
  artists_dic[artist] = approximate_playtime(sessions_merged[sessions_merged['name'] == artist]).sum()

artists_dic = {k: v for k, v in sorted(artists_dic.items(), key=lambda item: item[1])}

## Popularni wykonawcy

In [ ]:
print(list(artists_dic.keys())[-5:])

In [ ]:
TARGET_ARTISTS = ["Imagine Dragons", "Glass Animals", "Ed Sheeran"]

plot_actions(TARGET_ARTISTS, sessions_merged)

## Mniej popularni

In [ ]:
print(list(artists_dic.keys())[:5])

In [ ]:
TARGET_ARTISTS = ["Josh White", "Leroy Carr", "The Kooks"]

plot_actions(TARGET_ARTISTS, sessions_merged)

### Inne przydatne kolumny

Warto wziąć pod uwagę niektóre z cech utorów takie jak długość utworu, jego popularność i explicit kontent, które mogą dodatkowo wpływać na ilość odtworzeń / ostateczne wynagrodzenie artysty.

### Popularność

In [ ]:
print("Most popular:")
print(tracks.sort_values(by='popularity', ascending=False)[['name', 'popularity']].head(10))
print("=============================================")
print("Least popular:")
print(tracks.sort_values(by='popularity', ascending=True)[['name', 'popularity']].head(10))
print("=============================================")

# Wnioski końcowe

### Dane wejściowe i wyjściowe modelu

Na powyższych wykresach widać tendencje wzrostową / spadkową dla interakcji użytkowników z utowrami muzycznymi danych artystów. Po obrobieniu danych nasz model będzie przyjmować na wejście sekwencję reprezentującą łączny czas słuchania wykonawcy w kolejnych interwałach czasowych i na tej podstawie przewidywać średni czas słuchania wykonawcy w następnym interwale (przyszłości).